In [36]:
#### Dependencies

from PIL import Image # load image
from skimage.feature import greycomatrix, greycoprops

import glob
import math
import matplotlib.pyplot as plt # print image
import os


In [37]:
#### Global variables

model = {}
# dict: [
#          'cap' => [(contrast, correlation, energy, homogenity), ....],
#          'tulis' => [(contrast, correlation, energy, homogenity), ....]
#       ]

map_8bit_to_3bit = [i // 32 for i in range(256)]

# 0 - 255 -> 0 - 7

# (red, green, blue)
# 0 - 31  -> 0
# 32 - 63 -> 1
# 64 - 95 -> 2 ... sampai 7


In [59]:
#### Functions

def load_img(img_path):
    # load gambar
    return Image.open(img_path).convert('L')

def get_img_size(img):
    # ngambil (width, height)
    return img.size

def print_img(img):
    # print gambar
    plt.imshow(img, cmap='gray')
    
def get_resized_img(img, dimension):
    return img.resize(dimension)
    
def get_img_colors(img):
    return list(img.getdata())

def get_3bit_img_colors(img):
    img_colors = get_img_colors(img)
    
    loop_count = 0
    for img_color in img_colors:
        img_colors[loop_count] = map_8bit_to_3bit[img_color]
        
        loop_count += 1
        
    return img_colors

def get_img_matrix(img_colors):
    img_matrix = []
    
    loop_count = 0
    img_square_dimension = int(math.sqrt(len(img_colors)))
    
    for row in range(img_square_dimension):
        temp_row = []
        for col in range(img_square_dimension):
            temp_row.append(img_colors[loop_count])
            
            loop_count += 1
        img_matrix.append(temp_row)
        
    return img_matrix

def get_img_features(img):
    img_3bit_colors = get_3bit_img_colors(img)
    img_matrix = get_img_matrix(img_3bit_colors)
    
    glcm_matrix = greycomatrix(img_matrix, distances=[1], angles=[0], levels=12, symmetric=False, normed=False)
    glcm_components = ['contrast', 'correlation', 'energy', 'homogeneity', 'ASM', 'dissimilarity']
    
    img_features = []
    for glcm_component in glcm_components:
        img_features.append(greycoprops(glcm_matrix, glcm_component)[0][0])
        
    return tuple(img_features)

# Modelling

def init_model():
    global model
    
    model = {}
    
def get_class_names(training_path):
    return os.listdir(training_path)

def add_class_names_to_model(class_names):
    global model
    
    for class_name in class_names:
        if(not class_name in model):
            model[class_name] = []
            
def insert_img_features_to_model(class_name, img_features):
    global model
    
    if(class_name in model):
        model[class_name].append(img_features)

def load_preprocessed_img(img_path, dimension=(128, 128)):
    img = load_img(img_path)
    img = get_resized_img(img, dimension)
    
    return img
        
def train(training_path, img_type='*.jpg'):
    print('Training...')
    class_names = get_class_names(training_path)
    for class_name in class_names:
        img_paths = glob.glob(training_path + class_name + '/' + img_type)
        
        for img_path in img_paths:
            img = load_preprocessed_img(img_path)
            img_features = get_img_features(img)
            
            insert_img_features_to_model(class_name, img_features)
            
    print('--> Done')
    
def get_euclidian_distance(model_img_features, img_features):
    distance = 0.0
    
    for glcm_index in range(6):
        distance += (model_img_features[glcm_index] - img_features[glcm_index]) ** 2
        
    return math.sqrt(distance)
    
def get_img_class(img, k_neighbors=1):
    model_img_distances = []
    img_features = get_img_features(img)
    
    for class_name in model:
        for model_img_features in model[class_name]:
            euclidian_distance = get_euclidian_distance(model_img_features, img_features)
            model_img_distances.append((class_name, euclidian_distance))
            
    model_img_distances = sorted(model_img_distances, key= lambda x: x[1])
    
    top_class_names = []
    for neighbor_index in range(k_neighbors):
        top_class_names.append(model_img_distances[neighbor_index][0])
    
    img_class_name = max(set(top_class_names), key=top_class_names.count)
    
    return img_class_name
        
def validate(validation_path, img_type='*.jpg', k_neighbors=1):
    print('Validating...')
    class_names = get_class_names(training_path) # ['cap', 'tulis']
    total_correct_answer = 0
    total_guess = 0
    
    for class_name in class_names:
        img_paths = glob.glob(training_path + class_name + '/' + img_type)
        for img_path in img_paths:
            img = load_preprocessed_img(img_path)
            img_class_name = get_img_class(img)
            
            if(class_name == img_class_name):
                total_correct_answer += 1
                
            total_guess += 1
            
    print('--> Done')
    
    accuracy = (total_correct_answer / total_guess) * 100
    
    return accuracy
    

In [64]:
#### Main

# Defining paths
root_path       = './'
training_path   = root_path + 'training/'
validation_path = root_path + 'validation/'
test_path       = root_path + 'test/'

# Props
img_type = '*.jpg'
k_neighbors = 3

# Init
init_model()
class_names = get_class_names(training_path)
add_class_names_to_model(class_names)

train(training_path, img_type)

accuracy = validate(validation_path, img_type, k_neighbors)

print('Akurasi: ' + str(accuracy) + ' %')

Training...
--> Done
Validating...
--> Done
Akurasi: 100.0 %


In [47]:
img = load_img(test_path + 'tulis.jpg')
print(get_img_class(img, 1))

tulis
